In [1]:
import findspark

In [2]:
findspark.init('/home/bharath/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [5]:
df = spark.read.csv('smsspamcollection/SMSSpamCollection',sep='\t',inferSchema=True)

In [6]:
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



## Create a new feature called Length

In [7]:
from pyspark.sql.functions import length

In [8]:
df = df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [9]:
df = df.withColumn('length',length(df['text']))

In [10]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [11]:
df.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Preprocessing

In [12]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [13]:
tokenizer = Tokenizer(inputCol='text',outputCol='tokens')
stop_removed = StopWordsRemover(inputCol='tokens',outputCol='stop_tokens')
count = CountVectorizer(inputCol='stop_tokens',outputCol='count_vec')
idf = IDF(inputCol='count_vec',outputCol='idf_values')
ham_spam_numeric = StringIndexer(inputCol='class',outputCol='label')

## convert features into spark format

In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=['idf_values','length'],outputCol='features')

## Create a pipeline to convert the text to numeric values

In [16]:
from pyspark.ml import Pipeline

In [17]:
data_pipe = Pipeline(stages=[ham_spam_numeric,tokenizer,stop_removed,count,idf])

In [18]:
final = data_pipe.fit(df).transform(df)

In [19]:
output = assembler.transform(final)

## train the model

In [20]:
from pyspark.ml.classification import NaiveBayes

In [21]:
nb = NaiveBayes()

In [22]:
data = output.select('features','label')

In [23]:
train,test = data.randomSplit([0.8,0.2])

In [24]:
nb_fit = nb.fit(train)

In [25]:
test_results = nb_fit.transform(test)

## Evaluate the model

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [27]:
acc_eval = MulticlassClassificationEvaluator()

In [28]:
acc = acc_eval.evaluate(test_results)

In [29]:
acc

0.9310025089901958